# Performance by product --> SoM Dashboard

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Corner/Brands/data/'
CPG = 'colgate' 
nielsen_cat = False #True JUST FOR RB_HYHO, DANONE, UNILEVER SO FAR!!!
past = False
SOM_CPG = CPG.title()

## Import

In [2]:
import sys
sys.path.append('/Users/efraflores/Desktop/hub/cornershop/venv/lib/python3.9/site-packages')

In [3]:
import os
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
print([x for x in os.listdir(BASE_DIR) if x.endswith(f'{CPG}.csv')])

['nielsen_hills_science.csv', 'products_hills_science.csv', 'pasillos_hills_science.csv']


In [4]:
df = pd.read_csv(os.path.join(BASE_DIR,f'products_{CPG}.csv'),
                 sep='\t', encoding='UTF-16')
df.sample(2)

/Users/efraflores/Desktop/hub/cornershop/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (13,15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,month,country,city name,store_name,category en,category_id,product id,product name,barcodes,package,...,sales,orders,avg ticket,users,units found,units replaced,units requested,found rate,fulfillment,frequency
68877,abril 2021,MX,Toluca,Petco,Food,614,347008,Erizos e Insectivoros 1kg,{7501846401097},1KG,...,345,1,345,1,1,0,1,100.0%,100.0%,1.00
71671,marzo 2021,MX,Ciudad de México,Chedraui,Wet food,226,1001644,Alimento húmedo sabor sardina,{7501072209832},85 g,...,"1,000",25,40,25,84,0,84,100.0%,100.0%,1.00


In [5]:
aux_ini = df.copy()
aux_ini['sales'] = aux_ini['sales'].astype(str).str.replace(',','').astype(float)
aux_ini = aux_ini.groupby('month  ')['sales'].sum().reset_index()
aux_ini

,month,sales
0,abril 2021,21299124.00
1,julio 2021,21920943.00
2,junio 2021,21063065.00
3,marzo 2021,23183508.00
4,mayo 2021,21695710.00


In [6]:
import re
nielsen = pd.read_csv(BASE_DIR + f'nielsen_{CPG}.csv',
                      sep='\t',encoding='utf-16'
                     ).drop_duplicates(subset=['UPC','PRODUCT ID'])
display(nielsen['UPC'].head())
nielsen['UPC'] = nielsen['UPC'].apply(lambda x: re.sub('[^\d]','',str(x)))
nielsen['UPC'] = nielsen['UPC'].astype(str).str.strip().str.split().str[0].fillna(0).astype(int)
display(nielsen['UPC'].head())
display(nielsen['UPC'].head())
nielsen.sample(2)

0    7502281121649
1    7501072200723
2    7501777001151
3    7501072200730
4    7501072207098
Name: UPC, dtype: object

0    7502281121649
1    7501072200723
2    7501777001151
3    7501072200730
4    7501072207098
Name: UPC, dtype: int64

0    7502281121649
1    7501072200723
2    7501777001151
3    7501072200730
4    7501072207098
Name: UPC, dtype: int64

,UPC,FORMATO,PROVEEDOR GENERAL,PROVEEDOR ABIERTO,MARCA DETALLE,MARCA GENERICA,GRAMAJE,DESCRIPCION NIELSEN,PRODUCT ID,SEGMENTO,SUBSEGMENTO 1,SUBSEGMENTO 2,SUBSEGMENTO 3
777,7502002870399,NaN,ADM,NaN,NaN,Full-trust,20 kg,FullTrust Adulto 20 kg,391735,Dry food,NaN,NaN,NaN
246,7501253626427,NaN,Chedraui,NaN,NaN,Chedraui,454 g,Cacahuate tostado,168334,"Nuts, Seeds and Dried Fruits",NaN,NaN,NaN


## Functions

In [7]:
import time
import numpy as np
from IPython.lib.display import Audio

start = time.time()
def time_exp(x):
    minutes, seconds = np.floor(x/60), 60*(x/60-np.floor(x/60))
    print(f"{'{:.0f}'.format(minutes)} minutos con {'{:.2f}'.format(seconds)} segundos")
    
def tono(a = 1000, b = 700, play_time_seconds = 1, framerate = 4410):
    t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)*np.pi
    return Audio(np.sin(a*t) + np.sin(b*t), rate = framerate, autoplay=True)

In [8]:
def limpio(data,month_col,to_drop):
    df = data.copy()
    df[['mes','anio']] = df[month_col].str.split(expand=True)
    meses = dict(zip(['enero','febrero','marzo','abril','mayo','junio','julio',
                      'agosto','septiembre','octubre','noviembre','diciembre'],range(1,13)))
    df['mes'] = df['mes'].replace(meses)
    df.drop(columns=to_drop,inplace=True)
    df.columns = [str(x).replace('.','').replace(' ','_').lower() for x in df.columns]
    for col in ['brand_filter','cpg_filter']:
        df[f'{col[:-7]}_id'] = df[col].apply(lambda x: ','.join(re.findall('\s\((\d+)\)',str(x))))
        df[f'{col[:-7]}_name'] = df[col].apply(lambda x: re.sub('\s\(\d+\)','',str(x)))
    return df

## Transform

### Barcodes

In [9]:
df['barcodes'] = df['barcodes'].str.replace('{','').str.replace('}','')
df['barcodes'] = df['barcodes'].str.split(',',n=1).fillna('.').apply(lambda x:x[0])
df['barcodes'].sample(4)

/var/folders/j4/1p33_bh96yn8pdz3b_5t80hm0000gn/T/ipykernel_5164/2845833164.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['barcodes'] = df['barcodes'].str.replace('{','').str.replace('}','')


51457    7503008553040
10868    7500525079800
3913     7502268920678
20832      96619184736
Name: barcodes, dtype: object

### Merge

In [10]:
df = df.astype(str).merge(nielsen.astype(str).rename(columns={'UPC':'barcodes',
                                                              'PRODUCT ID':'product id'}),
                          on=['barcodes'] if nielsen_cat else ['barcodes','product id'],
                          how='left' if nielsen_cat else 'left')
if nielsen_cat:
    df = df.rename(columns={'product id_x':'product_id'}).drop(columns='product id_y')

df.sample(2)

,month,country,city name,store_name,category en,category_id,product id,product name,barcodes,package,...,PROVEEDOR GENERAL,PROVEEDOR ABIERTO,MARCA DETALLE,MARCA GENERICA,GRAMAJE,DESCRIPCION NIELSEN,SEGMENTO,SUBSEGMENTO 1,SUBSEGMENTO 2,SUBSEGMENTO 3
10271,julio 2021,MX,Mérida,Chedraui,"Nuts, Seeds and Dried Fruits",1004,2133238,Cacahuates estilo japonés,7500366003316,150 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20896,junio 2021,MX,Ciudad de México,Dogland,Dry food,230,1003019,Adulto raza pequeña,7503008553279,8kg,...,Nupec MX,nan,nan,Nupec,8kg,Adulto raza pequeña,Dry food,nan,nan,nan


### CPG

In [11]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('month  ')['sales'].sum().reset_index()
aux = aux_ini.copy()
aux = aux.set_index('month  ').join(aux_fin.set_index('month  '),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1
aux

,sales,sales_fin,dif,dif_p
month,,,,
abril 2021,21299124.00,21299124.00,0.00,0.00
julio 2021,21920943.00,21920943.00,0.00,0.00
junio 2021,21063065.00,21063065.00,0.00,0.00
marzo 2021,23183508.00,23183508.00,0.00,0.00
mayo 2021,21695710.00,21695710.00,0.00,0.00


In [12]:
prod_cpg = pd.read_csv(os.path.join(BASE_DIR,'pbbc.csv'),
                       #sep='\t',encoding='utf-16'
                      )
prod_cpg.sample(4)

,n,product_id,barcodes,category_id,category_name,brand_id,brand_name,cpg_id,cpg_name
100369,1,2676038,41220995172,979,Tortillas,9992,HEB,1074,HEB
79534,1,1214463,7501158411067,27,Leche,5861,San Marcos,1042,San Marcos
44070,1,329552,7501059947870,1253,Ollas y sartenes,4534,Ekco,757,Vasconia
98929,1,2436193,7500435158619,985,Detergentes para loza,5852,Salvo,1,P&G MX


In [13]:
prod_cpg.drop_duplicates(('barcodes' if nielsen_cat else 'product_id'),inplace=True)
prod_cpg = prod_cpg.astype(str).rename(columns={'product_id':'product id'})
prod_cpg['cpg filter'] = prod_cpg['cpg_name']+" ("+prod_cpg['cpg_id'].astype(str)+")"
prod_cpg.sample(4)

,n,product id,barcodes,category_id,category_name,brand_id,brand_name,cpg_id,cpg_name,cpg filter
81632,1,1235864,7501058617736,287,Chocolate,28,Nestlé,3,Nestlé MX,Nestlé MX (3)
41761,1,320120,7500435011280,973,Afeitado,447,Gillette,1,P&G MX,P&G MX (1)
101927,1,2916362,8411845017764,455,Rodados,13404,Famosa,1181,Famosa,Famosa (1181)
61935,1,450518,735257000014,1242,Gelatinas y flanes,10721,D'Gari,402,D´Gari,D´Gari (402)


In [14]:
df = df.merge(prod_cpg[[('barcodes' if nielsen_cat else 'product id'),'cpg filter']],
              on=('barcodes' if nielsen_cat else 'product id'),how='left')

In [15]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('month  ')['sales'].sum().reset_index()
aux = aux_ini.copy()
aux = aux.set_index('month  ').join(aux_fin.set_index('month  '),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1
aux

,sales,sales_fin,dif,dif_p
month,,,,
abril 2021,21299124.00,21299124.00,0.00,0.00
julio 2021,21920943.00,21920943.00,0.00,0.00
junio 2021,21063065.00,21063065.00,0.00,0.00
marzo 2021,23183508.00,23183508.00,0.00,0.00
mayo 2021,21695710.00,21695710.00,0.00,0.00


### Limpio

In [16]:
df = limpio(df,'month  ',
            ['month  ','country ','avg ticket','found rate','fulfillment','frequency'])
df.sample(2)

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_filter,...,subsegmento_1,subsegmento_2,subsegmento_3,cpg_filter,mes,anio,brand_id,brand_name,cpg_id,cpg_name
1132,Cancún,Soriana,Wet food,226,321662,Alimento paté salmón,50000290482,156 g,nan,Felix (2785),...,nan,nan,nan,Purina MX (561),7,2021,2785,Felix,561,Purina MX
51607,Toluca,Petco,Food,614,427418,Bandits premios hurón tocino,766501003826,85 g,MPFFD382,Marshall Pet (15734),...,NaN,NaN,NaN,NaN,5,2021,15734,Marshall Pet,,nan


In [17]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('mes')['sales'].sum().reset_index()
aux_fin['mes'] = aux_fin['mes'].astype(int).map(dict(zip(range(1,13),
                                               ['ene','feb','mar','abr','may','jun',
                                                'jul','ago','sep','oct','nov','dic'])))
aux = aux_ini.copy()
aux['month  '] = aux['month  '].str.split().str[0].str[:3]
aux = aux.set_index('month  ').join(aux_fin.set_index('mes'),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1
aux

,sales,sales_fin,dif,dif_p
month,,,,
abr,21299124.00,21299124.00,0.00,0.00
jul,21920943.00,21920943.00,0.00,0.00
jun,21063065.00,21063065.00,0.00,0.00
mar,23183508.00,23183508.00,0.00,0.00
may,21695710.00,21695710.00,0.00,0.00


In [18]:
aux = df[df['formato'].astype(str)=='nan'].copy()
df = df[~(df['formato'].astype(str)=='nan')].copy()
null_fill = {'proveedor_general':'cpg_name',
             'proveedor_abierto':'cpg_name',
             'marca_detalle':'brand_name',
             'marca_generica':'brand_name',
             'gramaje':'package',
             'descripcion_nielsen':'product_name',
             'segmento':'category_en'}
for x,y in null_fill.items():
    aux.loc[:,x] = aux.loc[:,y]
    
df = df.append(aux,ignore_index=True)

In [19]:
for col in df.columns:
    df[col] = df[col].astype(str).str.replace('nan','')
    
df = df.drop(columns='cpg_name').fillna('')
df.sample()

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_filter,...,segmento,subsegmento_1,subsegmento_2,subsegmento_3,cpg_filter,mes,anio,brand_id,brand_name,cpg_id
15258,Querétaro,HEB,Pet care & grooming,228,199490,Cloro Mascotas,41220969746,3.75 l,364218,HCF (10001),...,Pet care & grooming,,,,Hill Country Fare (1140),7,2021,10001,HCF,1140


In [20]:
brand_cols = ['formato','proveedor_general','proveedor_abierto','marca_detalle','marca_generica','gramaje','descripcion_nielsen','segmento','subsegmento_1','subsegmento_2','subsegmento_3']
display(df[brand_cols].sample())

for col in brand_cols:
    df[col] = df[col].str.strip().str.title()

display(df[brand_cols].sample())

,formato,proveedor_general,proveedor_abierto,marca_detalle,marca_generica,gramaje,descripcion_nielsen,segmento,subsegmento_1,subsegmento_2,subsegmento_3
76422,,,,Wet food (C-226),Wet food (C-226),100gr,Stuzzy Pouch Para Gato De Jamón,Wet food,,,


,formato,proveedor_general,proveedor_abierto,marca_detalle,marca_generica,gramaje,descripcion_nielsen,segmento,subsegmento_1,subsegmento_2,subsegmento_3
82733,,Mars Mx Pet,Mars Mx Pet,Pedigree,Pedigree,625 G,Alimento Húmedo Pollo,Wet Food,,,


In [21]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('mes')['sales'].sum().reset_index()
aux_fin['mes'] = aux_fin['mes'].astype(int).map(dict(zip(range(1,13),
                                               ['ene','feb','mar','abr','may','jun',
                                                'jul','ago','sep','oct','nov','dic'])))
aux_fin

,mes,sales
0,mar,23183508.00
1,abr,21299124.00
2,may,21695710.00
3,jun,21063065.00
4,jul,21920943.00


## Export

### Same product_id as past

In [22]:
print(len(df))
if past:
    som = pd.ExcelFile(os.path.join('/'.join(BASE_DIR.split('/')[:-2]),
                                    f'SoM_{SOM_CPG}.xlsx')).parse(sheet_name=0)
    df = df.merge(som.drop_duplicates('product_id')[['product_id']].reset_index(drop=True).astype(str))
    
print(len(df))

86317
86317


In [23]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('mes')['sales'].sum().reset_index()
aux_fin['mes'] = aux_fin['mes'].astype(int).map(dict(zip(range(1,13),
                                               ['ene','feb','mar','abr','may','jun',
                                                'jul','ago','sep','oct','nov','dic'])))

In [24]:
aux = aux_ini.copy()
aux['month  '] = aux['month  '].str.split().str[0].str[:3]
aux = aux.set_index('month  ').join(aux_fin.set_index('mes'),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1

### Ordered columns

In [25]:
export = df[['city_name', 'store_name', 'category_en', 'category_id', 'product_id',
             'product_name', 'barcodes', 'package', 'product_sku', 'brand_name',
             'sales', 'orders', 'users', 'units_found', 'units_replaced',
             'units_requested', 'formato', 'proveedor_general', 'proveedor_abierto',
             'marca_detalle', 'marca_generica', 'gramaje', 'descripcion_nielsen',
             'segmento', 'subsegmento_1', 'subsegmento_2', 'subsegmento_3',
             'mes', 'anio']].copy()
export.sort_values(['anio','mes'],inplace=True)
export.sample(4)

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_name,...,marca_detalle,marca_generica,gramaje,descripcion_nielsen,segmento,subsegmento_1,subsegmento_2,subsegmento_3,mes,anio
24458,Ciudad de México,Sumesa,"Nuts, Seeds and Dried Fruits",1004,335228,Cacahuate tostado sazonado,7501011185142,180 g,,Mafer,...,Mafer,Mafer,180 G,Cacahuate Tostado Sazonado,"Nuts, Seeds And Dried Fruits",,,,6,2021
48951,Querétaro,City Market,"Nuts, Seeds and Dried Fruits",1004,339396,Arándano deshidratado,7501071304477,Bolsa 75 g,,Verde Valle,...,Verde Valle,Verde Valle,Bolsa 75 G,Arándano Deshidratado,"Nuts, Seeds And Dried Fruits",,,,5,2021
30133,Monterrey,Petco,Treats,229,391892,Un Dos Treats Premio Sabor Res Y Moras 155 Gr,664697096809,Res y Moras 155 Gr,125395,Un dos treats,...,Un Dos Treats,Un Dos Treats,Res Y Moras 155 Gr,Un Dos Treats Premio Sabor Res Y Moras 155 Gr,Treats,,,,6,2021
66237,Querétaro,Costco,"Nuts, Seeds and Dried Fruits",1004,152333,Almendras tostadas,96619723829,1.13 kg,812369,Kirkland Signature,...,Kirkland Signature,Kirkland Signature,1.13 Kg,Almendras Tostadas,"Nuts, Seeds And Dried Fruits",,,,4,2021


In [26]:
export.to_csv(os.path.join(BASE_DIR,f'som_{CPG}.csv'),index=False,sep='\t',encoding='utf-16')

## Fin

In [27]:
aux

,sales,sales_fin,dif,dif_p
month,,,,
abr,21299124.00,21299124.00,0.00,0.00
jul,21920943.00,21920943.00,0.00,0.00
jun,21063065.00,21063065.00,0.00,0.00
mar,23183508.00,23183508.00,0.00,0.00
may,21695710.00,21695710.00,0.00,0.00


In [28]:
time_exp(time.time() - start)
tono()

0 minutos con 8.03 segundos
